<a href="https://colab.research.google.com/github/chethanmo/chethanmo.github.io/blob/master/ath.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install pandas pandas-ta numpy matplotlib yfinance
!pip install --upgrade pandas-datareader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pandas_ta as ta
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import yfinance as yf
from dateutil.relativedelta import relativedelta
from datetime import date as date


In [6]:
ticker_yahoo = yf.Ticker('HINDZINC.NS')
# ticker_yahoo.fast_info

file_path = '/content/EQUITY_L.csv'
fields = ['SYMBOL']
tickers = pd.read_csv(file_path, usecols=fields)


In [7]:
##lazy-loading dict with keys = ['currency', 'exchange', 'timezone', 'shares', 'market_cap', 'last_price', 'previous_close', 'open', 'day_high', 'day_low', 'regular_market_previous_close', 'last_volume', 'fifty_day_average', 'two_hundred_day_average', 'ten_day_average_volume', 'three_month_average_volume', 'year_high', 'year_low', 'year_change']

In [8]:
from concurrent.futures import ThreadPoolExecutor

ath_list = []
for i in range(0,len(tickers)):
  ath_list.append(tickers['SYMBOL'][i] +'.NS')

unwanted = {'INDO-RE.NS','CDSL.NS','HERIT-RE.NS','ZODIAC.NS','EMAMIPAP.NS','DIGJAMLMTD.NS','GUJRAFFIA.NS','NIRAJISPAT.NS','KALYANI.NS','ARTEMISMED.NS'}
ath_list = [e for e in ath_list if e not in unwanted]

#print (ath_list)

def get_stats(ticker):
    info = yf.Tickers(ticker).tickers[ticker].fast_info
    return ticker, info['last_price'], info['year_high'], info['two_hundred_day_average']
def paralel_func(ticker):
    with ThreadPoolExecutor(max_workers=None) as executor:
        results = list(executor.map(get_stats, ticker))
    return results

results = paralel_func(ath_list)


In [9]:
ticker_list = pd.DataFrame(results, columns=['Ticker', 'Last_price', 'ATH', 'two_EMA']) #creating the dataframe

pd.set_option('display.max_rows', None)

ticker_list = ticker_list.query('Last_price < ATH * 0.95 and Last_price > ATH * 0.9 and Last_price > two_EMA')
ticker_list.reset_index(drop=True, inplace=True)

# ticker_list.to_csv('Watchlist.csv', index=False)
#ticker_list


In [10]:
startdate = date.today() - relativedelta(years=4)
enddate = date.today() + relativedelta(days=1)

x = ticker_list['Ticker'].to_string(header=False,index=False).split('\n')
tickers = [','.join(ele.split()) for ele in x]

df = yf.download(tickers, start= startdate, end= enddate)

[*********************100%***********************]  116 of 116 completed

1 Failed download:
- BHARATFORG.NS: CircuitBreaker 'redis' is OPEN and does not permit further calls


In [18]:
pd.set_option('display.max_rows', 10)

ax_df = df.copy()

df1 = pd.DataFrame()


for ticker in tickers:
  test = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=10, multiplier=1)
  test1 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=11, multiplier=2)
  test2 = ta.supertrend(close=ax_df[('Close', ticker)], high=ax_df[('High', ticker)], low=ax_df[('Low', ticker)], length=12, multiplier=3)
  test3 = ta.ema(close=ax_df[('High', ticker)], length=32).to_frame('EMA200_H')
  test4 = ta.ema(close=ax_df[('Close', ticker)], length=32).to_frame('EMA200_C')
  test5 = ta.ema(close=ax_df[('Low', ticker)], length=32).to_frame('EMA200_L')
  test.columns = pd.MultiIndex.from_product([test.columns, [ticker]])
  test1.columns = pd.MultiIndex.from_product([test1.columns, [ticker]])
  test2.columns = pd.MultiIndex.from_product([test2.columns, [ticker]])
  test3.columns = pd.MultiIndex.from_product([test3.columns, [ticker]])
  test4.columns = pd.MultiIndex.from_product([test4.columns, [ticker]])
  test5.columns = pd.MultiIndex.from_product([test5.columns, [ticker]])
  df1 = pd.concat([df1, test, test1, test2, test3, test4, test5], axis=1)
  df1.sort_index()
  
ax_df = pd.concat([ax_df, df1], axis=1)
ax_df.reset_index()
ax_df = ax_df.drop(['Adj Close','Volume'], level=0, axis=1)


In [19]:
ax_df= ax_df.swaplevel(0, 1, 1).sort_index(axis=1)

In [20]:
ax_df= ax_df.tail(2)

In [21]:
from time import strftime
import datetime
prev_day = (date.today() - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
to_day = date.today().strftime('%Y-%m-%d')


In [22]:
df2= pd.DataFrame()
for ticker in tickers:
  cond1 = (ax_df.loc[to_day:,(ticker,'Close')] > ax_df.loc[to_day:,(ticker,'EMA200_H')]).bool()
  cond2 = (ax_df.loc[to_day:,(ticker,'Close')] > ax_df.loc[to_day:,(ticker,'Open')]).bool()
  cond3 = (ax_df.loc[prev_day:,(ticker,'SUPERTd_12_3.0')].to_numpy() < ax_df.loc[to_day:,(ticker,'SUPERTd_12_3.0')].to_numpy())
  #cond3 = (ax_df.loc[:,(ticker,'SUPERTd_10_1.0')] + ax_df.loc[:,(ticker,'SUPERTd_11_2.0')] + ax_df.loc[:,(ticker,'SUPERTd_12_3.0')] > -1).bool()

  if cond1 and cond2 and cond3[0]:
    ax_df1 = ax_df.loc[:,(ticker)]
    ax_df1.columns = pd.MultiIndex.from_product([[ticker],ax_df1.columns])
    df2 = pd.concat([df2,ax_df1], axis=1)
    
#df2.to_csv('test.csv')


In [23]:
pd.set_option('display.max_rows', None)
df2 = df2.T


In [32]:
def green_ticks(ax_df):
  green_tickers = pd.DataFrame()
  count = 0
  for ticker in tickers:
    cond1 = (ax_df.loc[to_day:,(ticker,'Close')] > ax_df.loc[to_day:,(ticker,'EMA200_H')]).bool()
    cond2 = (ax_df.loc[to_day:,(ticker,'Close')] > ax_df.loc[to_day:,(ticker,'Open')]).bool()
    cond3 = (ax_df.loc[prev_day:,(ticker,'Close')].to_numpy() < ax_df.loc[to_day:,(ticker,'Close')].to_numpy())
    cond4 = (ax_df.loc[to_day:,(ticker,'SUPERTd_10_1.0')] + ax_df.loc[to_day:,(ticker,'SUPERTd_11_2.0')] + ax_df.loc[to_day:,(ticker,'SUPERTd_12_3.0')] == 3).bool()
    cond5 = (ax_df.loc[prev_day:,(ticker,'SUPERTd_12_3.0')].to_numpy() < ax_df.loc[to_day:,(ticker,'SUPERTd_12_3.0')].to_numpy())
    cond6 = (ax_df.loc[to_day:,(ticker,'Close')] < 500).bool()

    #print(ticker, cond1, cond2, cond3)
  
    if cond1 and cond2 and cond3[0] and cond6 and cond4 or cond5[0]:
      count+=1
      ax_df2 = ax_df.loc[:,(ticker)]
      ax_df2['Change'] = (ax_df.loc[to_day:,(ticker,'Close')] - ax_df.loc[to_day:,(ticker,'Open')] * 100)/ax_df.loc[to_day:,(ticker,'Open')]
      ax_df2.columns = pd.MultiIndex.from_product([[ticker],ax_df2.columns])
      green_tickers = pd.concat([green_tickers,ax_df2], axis=1)

  return green_tickers,count

df3,count = green_ticks(ax_df)

print('Total number of uptrending stocks: ', count)
df3.T

<ipython-input-32-24adcd535381>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ax_df2['Change'] = (ax_df.loc[to_day:,(ticker,'Close')] - ax_df.loc[to_day:,(ticker,'Open')] * 100)/ax_df.loc[to_day:,(ticker,'Open')]


Total number of uptrending stocks:  6


Date                          2023-02-08  2023-02-09
BFINVEST.NS   Close           382.350006  433.700012
              EMA200_C        332.641507  338.766265
              EMA200_H        339.807494  345.940373
              EMA200_L        323.908976  327.369038
              High            386.399994  441.000000
              Low             363.549988  381.000000
              Open            366.549988  382.000000
              SUPERT_10_1.0   355.301319  387.293696
              SUPERT_11_2.0   336.175753  364.818875
              SUPERT_12_3.0   317.499583  343.314210
              SUPERTd_10_1.0    1.000000    1.000000
              SUPERTd_11_2.0    1.000000    1.000000
              SUPERTd_12_3.0    1.000000    1.000000
              SUPERTl_10_1.0  355.301319  387.293696
              SUPERTl_11_2.0  336.175753  364.818875
              SUPERTl_12_3.0  317.499583  343.314210
              SUPERTs_10_1.0         NaN         NaN
              SUPERTs_11_2.0         NaN         NaN
              SUPERTs_12_3.0         NaN         NaN
              Change                 NaN  -98.864660
CONTROLPR.NS  Close           464.799988  471.950012
              EMA200_C        429.670358  432.232761
              EMA200_H        438.211916  440.411194
              EMA200_L        422.284234  424.267008
              High            478.000000  474.500000
              Low             463.049988  455.000000
              Open            478.000000  455.000000
              SUPERT_10_1.0   452.394743  452.394743
              SUPERT_11_2.0   434.866734  434.866734
              SUPERT_12_3.0   417.891436  417.891436
              SUPERTd_10_1.0    1.000000    1.000000
              SUPERTd_11_2.0    1.000000    1.000000
              SUPERTd_12_3.0    1.000000    1.000000
              SUPERTl_10_1.0  452.394743  452.394743
              SUPERTl_11_2.0  434.866734  434.866734
              SUPERTl_12_3.0  417.891436  417.891436
              SUPERTs_10_1.0         NaN         NaN
              SUPERTs_11_2.0         NaN         NaN
              SUPERTs_12_3.0         NaN         NaN
              Change                 NaN  -98.962747
GEEKAYWIRE.NS Close            79.699997   95.599998
              EMA200_C         82.775693   83.552924
              EMA200_H         85.779851   86.375011
              EMA200_L         80.619060   80.520935
              High             81.000000   95.599998
              Low              78.449997   79.000000
              Open             80.699997   82.449997
              SUPERT_10_1.0    84.730252   81.135272
              SUPERT_11_2.0    89.888834   75.041967
              SUPERT_12_3.0    74.640255   69.002682
              SUPERTd_10_1.0   -1.000000    1.000000
              SUPERTd_11_2.0   -1.000000    1.000000
              SUPERTd_12_3.0    1.000000    1.000000
              SUPERTl_10_1.0         NaN   81.135272
              SUPERTl_11_2.0         NaN   75.041967
              SUPERTl_12_3.0   74.640255   69.002682
              SUPERTs_10_1.0   84.730252         NaN
              SUPERTs_11_2.0   89.888834         NaN
              SUPERTs_12_3.0         NaN         NaN
              Change                 NaN  -98.840509
INDHOTEL.NS   Close           318.450012  319.649994
              EMA200_C        310.681352  311.224906
              EMA200_H        314.818754  315.260042
              EMA200_L        305.833323  306.546455
              High            320.000000  322.100006
              Low             314.000000  317.600006
              Open            317.450012  319.000000
              SUPERT_10_1.0   309.346216  310.598225
              SUPERT_11_2.0   298.914974  301.374796
              SUPERT_12_3.0   288.875125  292.197137
              SUPERTd_10_1.0    1.000000    1.000000
              SUPERTd_11_2.0    1.000000    1.000000
              SUPERTd_12_3.0    1.000000    1.000000
              SUPERTl_10_1.0  309.346216  310.598225
              SUPERTl_11_

In [25]:
df2

""
